In [1]:
import numpy as np
import sklearn.metrics as metrics
import tensorflow as tf
import os, os.path
import pandas as pd
import math

import wide_residual_network_fix_v4 as wrn
from keras.datasets import cifar10
import keras.callbacks as callbacks
import keras.utils.np_utils as kutils
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

from keras import backend as K
from keras.backend.tensorflow_backend import set_session

ModuleNotFoundError: ignored

In [0]:
name = 'KaggleV2.3 - WRN-28-10'

if not os.path.exists(name):
    os.makedirs(name)

In [0]:
lr_schedule = [60, 120, 160] # epoch_step
def schedule(epoch_idx):
    if (epoch_idx + 1) < lr_schedule[0]:
        return 0.1
    elif (epoch_idx + 1) < lr_schedule[1]:
        return 0.02 # lr_decay_ratio = 0.2
    elif (epoch_idx + 1) < lr_schedule[2]:
        return 0.004
    return 0.0008

In [0]:

def random_crop(x, random_crop_size = (32,32), sync_seed=None):
    np.random.seed(sync_seed)
    w, h = x.shape[1], x.shape[2]
    rangew = (w - random_crop_size[0]) // 2
    rangeh = (h - random_crop_size[1]) // 2
    offsetw = 0 if rangew == 0 else np.random.randint(rangew)
    offseth = 0 if rangeh == 0 else np.random.randint(rangeh)
    return x[:, offsetw:offsetw+random_crop_size[0], offseth:offseth+random_crop_size[1]]

In [0]:
def pad(x, pad=4):
    return np.pad(x, ((0,0), (pad,pad),(pad,pad),(0,0)), mode='reflect')

In [0]:
batch_size = 64
nb_epoch = 200
img_rows, img_cols = 32, 32

(trainX, trainY), (testX, testY) = cifar10.load_data()

trainX = pad(trainX)
testX = pad(testX)

trainX = trainX.astype('float32')
trainX = (trainX - [125.3, 123.0, 113.9]) / [63.0, 62.1, 66.7]
testX = testX.astype('float32')
testX = (testX - [125.3, 123.0, 113.9]) / [63.0, 62.1, 66.7]


trainY = kutils.to_categorical(trainY)
testY = kutils.to_categorical(testY)

generator = ImageDataGenerator(zca_epsilon=0,
                               horizontal_flip=True,
                               fill_mode='reflect',)

generator.fit(trainX, seed=0, augment=True)

test_generator = ImageDataGenerator(zca_epsilon=0,
                               horizontal_flip=True,
                               fill_mode='reflect',)

test_generator.fit(testX, seed=0, augment=True)

In [0]:
init_shape = (3, 32, 32) if 0 == 'th' else (32, 32, 3)
model = wrn.create_wide_residual_network(init_shape, nb_classes=10, N=4, k=10, dropout=0.0)

model.summary()
# plot_model(model, "WRN-28-10.png", show_shapes=True)

opt = SGD(lr=0.0008, nesterov=True, decay=0.0005)

model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["acc"])
print("Finished compiling")
print("Allocating GPU memory")

model.load_weights('KaggleV2.3 - WRN-28-10/WRN-28-10 Weights.h5')
print("Model loaded.")

In [0]:
csv_logger = callbacks.CSVLogger(name + '/log.csv', append=True, separator=';')
save_c = callbacks.ModelCheckpoint(name + '/WRN-28-10 Weights.h5', monitor="val_acc", save_best_only=True)
lrs = LearningRateScheduler(schedule=schedule)

In [0]:
for i in range(0, nb_epoch):
    print('i = ' + str(i))
    for x_batch, y_batch in generator.flow(trainX, trainY, batch_size=len(trainX)):
        for testx_batch, testy_batch in test_generator.flow(testX, testY, batch_size=len(testX)):
            x_batch = random_crop(x_batch)
            testx_batch = random_crop(testx_batch)
            model.fit(x_batch, y_batch, nb_epoch=1, callbacks=[csv_logger, save_c, lrs], validation_data=(testx_batch, testy_batch))
            break
        break

scores = model.evaluate_generator(test_generator.flow(testX, testY, nb_epoch), (testX.shape[0] / batch_size + 1))
print("Accuracy = %f" % (100 * scores[1]))

In [0]:

def submitFile(modelFile, saveFile, model):
    # model.load_weights(modelFile)
    test_x = np.load('All images.npy')
    test_x[1] = (test_x[1] - test_x[1].mean(axis=0)) / (test_x[1].std(axis=0))
    bs = 100
    steps = math.ceil(int(test_x.shape[0])/bs)
    print ("Steps - " + str(steps))
    pred = model.predict_generator(test_generator.flow(test_x, batch_size=bs, shuffle=False), steps=steps)
    y_pred = np.argmax(pred, axis=1)
    labels = labelsToStrings(y_pred)
    labels.to_csv(saveFile)

In [0]:
def labelsToStrings(data):
    ret = []
    for label in data:
        temp = ''
        
        if(label == 0): temp = 'airplane'
        if(label == 1): temp = 'automobile'
        if(label == 2): temp = 'bird'
        if(label == 3): temp = 'cat'
        if(label == 4): temp = 'deer'
        if(label == 5): temp = 'dog'
        if(label == 6): temp = 'frog'
        if(label == 7): temp = 'horse'
        if(label == 8): temp = 'ship'
        if(label == 9): temp = 'truck'
        
        ret.append(temp)
    
    label_id = np.arange(data.shape[0]) + 1
    s = pd.Series(ret, index=label_id)
    s = s.to_frame('label')
    s.index.name='id'
    
    return s

In [0]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

baseMapNum = 32
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

#training
batch_size = 64
epochs=25
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=3*epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep75.h5')

opt_rms = keras.optimizers.rmsprop(lr=0.0005,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep100.h5')

opt_rms = keras.optimizers.rmsprop(lr=0.0003,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep125.h5')

#testing - no kaggle eval
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

Using TensorFlow backend.


170500096/170498071 [==============================] - 12s 0us/step
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
___________________